# Documentation du Dataset pour l'Environnement RL

Ce document décrit en détail la préparation du jeu de données issu des fichiers ATUS pour entraîner un agent Deep Q-Network (DQN) dans un environnement de gestion du temps.

---

## 1. Contexte et objectifs

L'objectif est de simuler un calendrier d'activités à partir de données réelles (American Time Use Survey) pour :

* Construire un **environnement Gym-like** exposant un état (créneaux, type d'activité, lieu, contexte) et acceptant des actions (allocation de tâches).
* Entraîner un **agent DQN** capable de proposer des plannings optimaux.
* Itérer via un **feedback utilisateur** pour améliorer la productivité.

---

## 2. Sources de données

Les fichiers bruts (`data/raw/`) :

1. **atusact.csv** : journal minute-par-minute des activités
2. **codes.csv** : correspondance `code` → `name` des activités
3. **atusresp.csv** : données démographiques et contexte du répondant
4. **atuswgts.csv** : poids d'échantillonnage pour chaque journal
5. **atussum.csv** : résumé agrégé journalier par catégorie

---

## 3. Colonnes retenues

Pour un proof-of-concept, seules les colonnes suivantes sont nécessaires :

| Colonne         | Origine               | Description                                                      |
| --------------- | --------------------- | ---------------------------------------------------------------- |
| `TUCASEID`      | atusact,resp,wgts,sum | Identifiant unique du journal (utilisateur+jour)                 |
| `TUACTIVITY_N`  | atusact               | Code numérique de l'activité                                     |
| `ACTIVITY_NAME` | codes                 | Libellé textuel de l'activité                                    |
| `TUACTDUR24`    | atusact               | Durée de l'activité en minutes                                   |
| `TUSTARTTIM`    | atusact               | Heure de début (format HHMM)                                     |
| `TEWHERE`       | atusact               | Code du lieu d'exécution de l'activité                           |
| `TUDIARYDAY`    | atusresp              | Jour de la semaine (1=Lundi … 7=Dimanche)                        |
| `TUFNWGTP001`   | atuswgts              | Poids d'échantillonnage principal                                |
| `TUFNWGTP002`   | atuswgts              | Poids secondaire (optionnel)                                     |
| `GEMETSTA`      | atussum               | Code d'état général du jour (semaine vs. week-end, etc.)         |
| `GTMETSTA`      | atussum               | Code de synthèse temporelle (type de jour métier vs. non-métier) |

In [ ]:
col_map = {
    'TUCASEID': 'user_id',
    'TUACTIVITY_N': 'activity_code',
    'ACTIVITY_NAME': 'activity_name',
    'TUACTDUR24': 'duration_minutes',
    'TUSTARTTIM': 'start_time',
    'TEWHERE': 'location_code',
    'TUDIARYDAY': 'day_of_week',
    'TEAGE': 'age',
    'TESEX': 'sex',
    'TUFNWGTP001': 'weight_main',
    'TUFNWGTP002': 'weight_secondary',
    'GEMETSTA': 'general_state',
    'GTMETSTA': 'temporal_state'
}
# Exemple d'application :
import pandas as pd

df = pd.read_csv('data/raw/donnees_nettoyees.csv')
df.rename(columns=col_map, inplace=True)

In [ ]:
# 1_data_exploration.ipynb
import pandas as pd
import matplotlib.pyplot as plt

# Chargement et renommage
col_map = { ... }  # cf. section commune
df = pd.read_csv('data/raw/donnees_nettoyees.csv')
df.rename(columns=col_map, inplace=True)

# Aperçu général
display(df.head())

df['activity_name'].value_counts().plot(kind='bar', title='Répartition des activités')
plt.xlabel('Activité')
plt.ylabel('Nombre d'observations')
plt.show()

# Statistiques simples
display(df.describe(include='all'))

In [3]:
# 1_data_exploration.ipynb
# Objectif : explorer le dataset nettoyé pour identifier les colonnes utiles

import pandas as pd

# Chargement du dataset nettoyé
df = pd.read_csv('../data/raw/donnees_nettoyees.csv')

# Colonnes du dataset
print("Colonnes disponibles :", df.columns.tolist())

# Statistiques basiques
print(df[['TUCASEID','TUACTIVITY_N','TUACTDUR24','TUSTARTTIM','TEWHERE']].describe())
print("Valeurs uniques pour DAY:", df['TUDIARYDAY'].unique())

# Vérifier la distribution des activités
activity_counts = df['ACTIVITY_NAME'].value_counts()
activity_counts.head(10)

Colonnes disponibles : ['TUCASEID', 'TUACTIVITY_N', 'TUACTDUR24', 'TUSTARTTIM', 'TEWHERE', 'ACTIVITY_NAME', 'TUDIARYDAY', 'TUFNWGTP001', 'TUFNWGTP002', 'GEMETSTA', 'GTMETSTA', 'ACTIVITY_NAME_CLEANED', 'ACTIVITY_NAME_Caring For & Helping Household (HH) Members', 'ACTIVITY_NAME_Caring for & Helping Nonhousehold (NonHH) Members', 'ACTIVITY_NAME_Consumer Purchases', 'ACTIVITY_NAME_Data Codes', 'ACTIVITY_NAME_Eating and Drinking', 'ACTIVITY_NAME_Education', 'ACTIVITY_NAME_Government Services & Civic Obligations', 'ACTIVITY_NAME_Household Activities', 'ACTIVITY_NAME_Household Services', 'ACTIVITY_NAME_Personal Care Activities', 'ACTIVITY_NAME_Professional & Personal Care Services', 'ACTIVITY_NAME_Religious and Spiritual Activities', 'ACTIVITY_NAME_Socializing, Relaxing, and Leisure', 'ACTIVITY_NAME_Sports, Exercise, & Recreation', 'ACTIVITY_NAME_Telephone Calls', 'ACTIVITY_NAME_Traveling', 'ACTIVITY_NAME_Volunteer Activities', 'ACTIVITY_NAME_Work & Work-Related Activities', 'TUACTDUR24_scale

ACTIVITY_NAME
Personal Care Activities                             170842
Household Activities                                 170842
Caring For & Helping Household (HH) Members          170842
Caring for & Helping Nonhousehold (NonHH) Members    170842
Work & Work-Related Activities                       170842
Education                                            169904
Consumer Purchases                                   168503
Professional & Personal Care Services                166347
Household Services                                   163210
Government Services & Civic Obligations              159192
Name: count, dtype: int64

In [ ]:
import pandas as pd
import os
import sys

# --- 0. Paramètres de chemin ---
DATA_RAW       = '../data/raw/sample_small'
DATA_PROCESSED = '../data/raw'
os.makedirs(DATA_PROCESSED, exist_ok=True)

ACT_PATH    = os.path.join(DATA_RAW, 'atusact.csv')
CODES_PATH  = os.path.join(DATA_RAW, 'codes.csv')
RESP_PATH   = os.path.join(DATA_RAW, 'atusresp.csv')
WGTS_PATH   = os.path.join(DATA_RAW, 'atuswgts.csv')
SUM_PATH    = os.path.join(DATA_RAW, 'atussum.csv')
OUTPUT_PATH = os.path.join(DATA_PROCESSED, 'atus_full_selected.csv')

# --- 1. Chargement des petits jeux (en mémoire) ---
try:
    codes = pd.read_csv(CODES_PATH, dtype=str)
    resp  = pd.read_csv(RESP_PATH, dtype=str)
    wgts  = pd.read_csv(WGTS_PATH, dtype=str)
    summ  = pd.read_csv(SUM_PATH, dtype=str)
except FileNotFoundError as e:
    print(f"Erreur : impossible de charger un fichier : {e}", file=sys.stderr)
    sys.exit(1)

# --- 2. Mise en majuscules des noms de colonnes pour homogénéité ---
for df in (codes, resp, wgts, summ):
    df.columns = df.columns.str.upper()

# --- 3. Renommage dans `codes` pour harmoniser ---
codes = codes.rename(columns={
    'CODE': 'TUACTIVITY_N',
    'NAME': 'ACTIVITY_NAME'
})

# --- 4. Définition des colonnes souhaitées ---
desired_resp = ['TUCASEID', 'TEAGE', 'TESEX', 'TUDIARYDAY']
desired_wgts = ['TUCASEID', 'TUFNWGTP001', 'TUFNWGTP002']
desired_summ = ['TUCASEID', 'GEMETSTA', 'GTMETSTA']

# 5. Intersection : ne garder que ce qui existe réellement
avail_resp = [c for c in desired_resp if c in resp.columns]
avail_wgts = [c for c in desired_wgts if c in wgts.columns]
avail_summ = [c for c in desired_summ if c in summ.columns]

if len(avail_resp) < len(desired_resp):
    missing = set(desired_resp) - set(avail_resp)
    print(f"⚠️  Colonnes RESP manquantes, ignorées: {missing}", file=sys.stderr)
if len(avail_wgts) < len(desired_wgts):
    missing = set(desired_wgts) - set(avail_wgts)
    print(f"⚠️  Colonnes WGTS manquantes, ignorées: {missing}", file=sys.stderr)
if len(avail_summ) < len(desired_summ):
    missing = set(desired_summ) - set(avail_summ)
    print(f"⚠️  Colonnes SUM manquantes, ignorées: {missing}", file=sys.stderr)

# On sélectionne uniquement ces colonnes dans les dataframes
resp = resp[avail_resp]
wgts = wgts[avail_wgts]
summ = summ[avail_summ]

# --- 6. Traitement du fichier atusact par chunks ---
# Colonnes d'intérêt dans atusact
act_cols = [
    'TUCASEID', 'TULINENO',
    'TUACTIVITY_N', 'TUACTDUR24',
    'TUSTARTTIM', 'TEWHERE'
]

chunksize   = 500_000
first_chunk = True

for chunk in pd.read_csv(ACT_PATH, dtype=str, chunksize=chunksize):
    # a) uniformiser en majuscules
    chunk.columns = chunk.columns.str.upper()
    # b) ne garder que les colonnes utiles si elles existent
    keep = [c for c in act_cols if c in chunk.columns]
    chunk = chunk[keep]
    # c) fusion code → description (si TUACTIVITY_N présent)
    if 'TUACTIVITY_N' in chunk.columns:
        chunk = chunk.merge(codes[['TUACTIVITY_N', 'ACTIVITY_NAME']],
                            on='TUACTIVITY_N', how='left')
    # d) fusion RESP, WGTS, SUM
    if 'TUCASEID' in chunk.columns:
        chunk = chunk.merge(resp, on='TUCASEID', how='left')
        chunk = chunk.merge(wgts, on='TUCASEID', how='left')
        chunk = chunk.merge(summ, on='TUCASEID', how='left')
    # e) écriture incrémentale dans le fichier de sortie
    chunk.to_csv(
        OUTPUT_PATH,
        mode='w' if first_chunk else 'a',
        index=False,
        header=first_chunk
    )
    first_chunk = False

print(f"✅ Dataset final prêt : {OUTPUT_PATH}")


⚠️  Colonnes RESP manquantes, ignorées: {'TESEX', 'TEAGE'}


✅ Dataset final prêt : data/raw\atus_full_selected.csv
